In [ ]:
!git clone https://github.com/Kolibri25/WINDIFY_AI


fatal: destination path 'WINDIFY_AI' already exists and is not an empty directory.


In [ ]:
cd WINDIFY_AI

/content/WINDIFY_AI


Setup environment

In [ ]:
!pip install -r requirements.txt

In [ ]:
import scripts.download_barra as dnb
import scripts.load_data as ld
import scripts.model_input as mi


Below purely to document BARRA data download and import procedure:

In [ ]:
'''

# download barrac2 data from months of Oct, Nov, Dec, Jan from 1979 to 2024
# lat/lon values and variables are hard-coded in the script

dnb.main()

Importing previously downloaded data from Google Drive shared folder

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# combine files into single dataset per variable
# BARRAc2
ld.load_and_save_barra_c2()
# ERA5
ld.load_and_save_era5()